Ahora vamos a desarrollar una función para poder entrenar la red, ya que hay que hacer el forward tanto para entrenamiento como para test vamos a utilizar la misma función.

Dicha función recibirá el modelo, el dataloader, el optimizador y si estamos entrenando o no. Debemos tener un bucle que itere sobre cada batch del dataloader, en cada iteración el dataloader devuelve una tupla que contiene los datos de entrada y la salida esperada.

Para cada tupla debemos poner a cero los gradientes de los pesos (si estamos entrenando). Después convertimos la entrada y el objetivo a un objeto llamado Variable dentro de autograd, esto hace que el objeto sea capaz de almacenar los gradientes y propagarlos. Una vez hecho esto podemos realizar el forward sobre la red y obtendremos la predicción (score) y el valor que devuelve la función de pérdida (loss).

Con el valor de la predicción calculamos la clase predicha simplemente eligiendo la que obtenga el mayor valor y después comprobamos cuantas han sido correctas.

En el caso de estar entrenando devemos propagar los gradientes con el algoritmo backward y después utilizar el optimizador para actualizar los pesos de la red. La función que acabamos de describir sería la siguiente:

In [ ]:
def evaluate(model, dataset_loader, optimizer, train=False):
    correct_cnt, ave_loss = 0, 0#Contador de aciertos y acumulador de la función de pérdida
    count = 0#Contador de muestras
    for batch_idx, (x, target) in enumerate(dataset_loader):
        count += len(x)#sumamos el tamaño de batch, esto es porque n_batches*tamaño_batch != n_muestras
        if train:
            optimizer.zero_grad()#iniciamos a 0 los valores de los gradiente
        x, target = Variable(x), Variable(target)#Convertimos el tensor a variable del modulo autograd
        score, loss = model(x, target)#realizamos el forward
        _, pred_label = torch.max(score.data, 1)#pasamos de one hot a número
        correct_cnt += (pred_label == target.data).sum()#calculamos el número de etiquetas correctas
        ave_loss += loss.data[0]#sumamos el resultado de la función de pérdida para mostrar después
        if train:
            loss.backward()#Calcula los gradientes y los propaga
            optimizer.step()#adaptamos los pesos con los gradientes propagados
    accuracy = correct_cnt/count#Calculamos la precisión total
    ave_loss /= count#Calculamos la pérdida media
    print ('==>>>loss: {:.6f}, accuracy: {:.4f}'.format(ave_loss, accuracy))#Mostramos resultados

Entrenar nuestro modelo ahora es muy sencillo, por ejemplo, si queremos realizar el proceso de entrenamiento durante 10 epochs testeando en cada una simplemente ejecutamos el siguiente código:

In [ ]:
for epoch in range(10):
    print("Epoch: {}".format(epoch))
    print("Train")
    evaluate(model, train_loader, optimizer, train=True)
    print("Test")
    evaluate(model, test_loader, optimizer, train=False)